In [1]:
import tensorflow as tf
import numpy as np
import pickle

/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [2]:
with open('dataset.pkl', 'rb') as fopen:
    dataset = pickle.load(fopen)

In [24]:
s = []

length = 1024

for l in range(len(dataset)):
    for i in range(0, len(dataset[l]), length):
        index = min(i + length, len(dataset[l]))
        x = dataset[l][i: index]
        if len(x) != length:
            x = dataset[l][index - length: index]
        s.append(x)

In [25]:
def create_int_feature(values):
    feature = tf.train.Feature(
        int64_list = tf.train.Int64List(value = list(values))
    )
    return feature

In [26]:
r = tf.python_io.TFRecordWriter('dataset.tfrecord')

In [27]:
import collections
from tqdm import tqdm

for i in tqdm(range(len(s))):
    features = collections.OrderedDict()
    features['input_ids'] = create_int_feature(s[i])
    tf_example = tf.train.Example(
        features = tf.train.Features(feature = features)
    )
    r.write(tf_example.SerializeToString())

100%|██████████| 824148/824148 [05:00<00:00, 2741.23it/s]


In [28]:
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'mesolitica-storage.json'

In [29]:
from google.cloud import storage
client = storage.Client()
bucket = client.bucket('mesolitica-general')

In [31]:
import glob

files = glob.glob('*.tfrecord')
files

['dataset.tfrecord']

In [33]:
from tqdm import tqdm

for file in tqdm(files):
    blob = bucket.blob(f'gpt2-data/{file}')
    blob.upload_from_filename(file)

100%|██████████| 1/1 [01:23<00:00, 83.47s/it]
